Horizon 1 LSTM

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from bayes_opt import BayesianOptimization
from statsmodels.tsa.seasonal import STL
from scipy.fftpack import fft
import warnings

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    df['month'] = df.index.month
    df['weekofyear'] = df.index.isocalendar().week
    df['year'] = df.index.isocalendar().year
    df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    fft_values = fft(df[target_col].dropna().values)
    fft_real = np.real(fft_values)[:len(df[target_col])]
    fft_imag = np.imag(fft_values)[:len(df[target_col])]
    df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # LSTM expects 3D input
    return X_scaled, y, scaler

# ============================================
# 3. Build LSTM Model
# ============================================
def build_lstm_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(num_units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Inference time per sample
    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Testing time
    start_test = time.time()
    model.evaluate(X_test, y_test, verbose=0)  # Just to trigger testing phase
    testing_time = time.time() - start_test

    # Evaluation
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation Results =====")
    print(f"Test Accuracy: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    # Save & calculate model size
    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)  # MB
    os.remove("temp_model.h5")

    # Trainable Parameters
    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])

    # RAM usage
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # MB

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Inference Time per Sample: {inference_time:.6f} seconds")
    print(f"Testing Time: {testing_time:.2f} seconds")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Parameters: {total_params:,}")
    print(f"RAM Usage During Inference: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 1.0       | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 58.87     | 0.237     | 99.91     | 32.34     |
| 7         | 1.0       | 63.83     | 0.1911    | 99.11     | 127.8     |
| 8         | 1.0       | 17.24     | 0.4906    | 50.56     | 33.63     |
| 9         | 1.0       | 16.41     | 0.4657    | 98.87     | 127.3     |
| 10        | 1.0       | 18.2      | 0.4567    | 99.88     | 32.67     |
| 11        | 1.0       | 63.94     | 0.4128    | 50.58     | 125.6     |
| 12   


===== Evaluation Results =====
Test Accuracy: 0.9680
Classification Report:
               precision    recall  f1-score   support

           0     1.0000    0.7143    0.8333        56
           1     0.9370    0.9917    0.9636       240
           2     0.9938    0.9856    0.9897       487
           3     0.8056    0.9667    0.8788        30

    accuracy                         0.9680       813
   macro avg     0.9341    0.9146    0.9163       813
weighted avg     0.9705    0.9680    0.9671       813

Confusion Matrix:
 [[ 40  16   0   0]
 [  0 238   2   0]
 [  0   0 480   7]
 [  0   0   1  29]]

===== Model Metrics =====
Training Time: 18.29 seconds
Inference Time per Sample: 0.000632 seconds
Testing Time: 0.11 seconds
Model Size: 0.20 MB
Trainable Parameters: 46,148
RAM Usage During Inference: 978.09 MB


Horizon 2 LSTM

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from bayes_opt import BayesianOptimization
from statsmodels.tsa.seasonal import STL
from scipy.fftpack import fft
import warnings

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h2'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    df['month'] = df.index.month
    df['weekofyear'] = df.index.isocalendar().week
    df['year'] = df.index.isocalendar().year
    df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    fft_values = fft(df[target_col].dropna().values)
    fft_real = np.real(fft_values)[:len(df[target_col])]
    fft_imag = np.imag(fft_values)[:len(df[target_col])]
    df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # LSTM expects 3D input
    return X_scaled, y, scaler

# ============================================
# 3. Build LSTM Model
# ============================================
def build_lstm_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(num_units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Inference time per sample
    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Testing time
    start_test = time.time()
    model.evaluate(X_test, y_test, verbose=0)  # Just to trigger testing phase
    testing_time = time.time() - start_test

    # Evaluation
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation Results =====")
    print(f"Test Accuracy: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    # Save & calculate model size
    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)  # MB
    os.remove("temp_model.h5")

    # Trainable Parameters
    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])

    # RAM usage
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # MB

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Inference Time per Sample: {inference_time:.6f} seconds")
    print(f"Testing Time: {testing_time:.2f} seconds")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Parameters: {total_params:,}")
    print(f"RAM Usage During Inference: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\train_lasso_selected.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\test_lasso_selected.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 1.0       | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 58.87     | 0.237     | 99.91     | 32.34     |
| 7         | 1.0       | 63.83     | 0.1911    | 99.11     | 127.8     |
| 8         | 1.0       | 17.24     | 0.4906    | 50.56     | 33.63     |
| 9         | 1.0       | 16.41     | 0.4657    | 98.87     | 127.3     |
| 10        | 1.0       | 18.2      | 0.4567    | 99.88     | 32.67     |
| 11        | 1.0       | 63.94     | 0.4128    | 50.58     | 125.6     |
| 12   


===== Evaluation Results =====
Test Accuracy: 0.9533
Classification Report:
               precision    recall  f1-score   support

           0     1.0000    0.2619    0.4151        42
           1     0.8721    0.9956    0.9298       226
           2     0.9960    0.9902    0.9931       509
           3     0.9211    0.9722    0.9459        36

    accuracy                         0.9533       813
   macro avg     0.9473    0.8050    0.8210       813
weighted avg     0.9585    0.9533    0.9435       813

Confusion Matrix:
 [[ 11  31   0   0]
 [  0 225   1   0]
 [  0   2 504   3]
 [  0   0   1  35]]

===== Model Metrics =====
Training Time: 22.64 seconds
Inference Time per Sample: 0.000780 seconds
Testing Time: 0.12 seconds
Model Size: 0.18 MB
Trainable Parameters: 42,588
RAM Usage During Inference: 1349.47 MB


Horizon 3 LSTM

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from bayes_opt import BayesianOptimization
from statsmodels.tsa.seasonal import STL
from scipy.fftpack import fft
import warnings

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h3'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()



    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # LSTM expects 3D input
    return X_scaled, y, scaler

# ============================================
# 3. Build LSTM Model
# ============================================
def build_lstm_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(num_units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Inference time per sample
    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Testing time
    start_test = time.time()
    model.evaluate(X_test, y_test, verbose=0)  # Just to trigger testing phase
    testing_time = time.time() - start_test

    # Evaluation
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation Results =====")
    print(f"Test Accuracy: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    # Save & calculate model size
    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)  # MB
    os.remove("temp_model.h5")

    # Trainable Parameters
    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])

    # RAM usage
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # MB

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Inference Time per Sample: {inference_time:.6f} seconds")
    print(f"Testing Time: {testing_time:.2f} seconds")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Parameters: {total_params:,}")
    print(f"RAM Usage During Inference: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H3_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H3_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9919    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9959    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9878    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9878    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9878    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9959    | 22.97     | 0.3185    | 53.09     | 116.6     |
| 7         | 0.9959    | 17.35     | 0.2868    | 66.53     | 109.0     |
| 8         | 1.0       | 19.85     | 0.259     | 96.99     | 127.5     |
| 9         | 0.9919    | 46.43     | 0.2861    | 99.9      | 126.2     |
| 10        | 0.9959    | 16.08     | 0.4653    | 96.12     | 110.0     |
| 11        | 1.0       | 16.06     | 0.2302    | 85.84     | 127.9     |
| 12   


===== Evaluation Results =====
Test Accuracy: 0.9766
Classification Report:
               precision    recall  f1-score   support

           0     1.0000    0.4815    0.6500        27
           1     0.9378    0.9906    0.9635       213
           2     0.9962    0.9943    0.9952       524
           3     0.9423    1.0000    0.9703        49

    accuracy                         0.9766       813
   macro avg     0.9691    0.8666    0.8947       813
weighted avg     0.9778    0.9766    0.9739       813

Confusion Matrix:
 [[ 13  14   0   0]
 [  0 211   2   0]
 [  0   0 521   3]
 [  0   0   0  49]]

===== Model Metrics =====
Training Time: 25.98 seconds
Inference Time per Sample: 0.000652 seconds
Testing Time: 0.20 seconds
Model Size: 0.33 MB
Trainable Parameters: 81,980
RAM Usage During Inference: 1699.04 MB


Horizon 4 LSTM

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from bayes_opt import BayesianOptimization
from statsmodels.tsa.seasonal import STL
from scipy.fftpack import fft
import warnings

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h4'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()



    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # LSTM expects 3D input
    return X_scaled, y, scaler

# ============================================
# 3. Build LSTM Model
# ============================================
def build_lstm_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(num_units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Inference time per sample
    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Testing time
    start_test = time.time()
    model.evaluate(X_test, y_test, verbose=0)  # Just to trigger testing phase
    testing_time = time.time() - start_test

    # Evaluation
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation Results =====")
    print(f"Test Accuracy: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    # Save & calculate model size
    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)  # MB
    os.remove("temp_model.h5")

    # Trainable Parameters
    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])

    # RAM usage
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # MB

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Inference Time per Sample: {inference_time:.6f} seconds")
    print(f"Testing Time: {testing_time:.2f} seconds")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Parameters: {total_params:,}")
    print(f"RAM Usage During Inference: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H4_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H4_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9919    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9919    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9878    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9837    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9878    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9919    | 22.97     | 0.3185    | 53.09     | 116.6     |
| 7         | 0.9959    | 16.26     | 0.3996    | 90.52     | 118.5     |
| 8         | 0.9959    | 28.92     | 0.1585    | 99.79     | 125.3     |
| 9         | 0.9959    | 16.01     | 0.4354    | 98.89     | 104.6     |
| 10        | 0.9959    | 17.0      | 0.2281    | 99.71     | 120.4     |
| 11        | 0.9919    | 62.65     | 0.3616    | 99.98     | 127.8     |
| 12   


===== Evaluation Results =====
Test Accuracy: 0.9582
Classification Report:
               precision    recall  f1-score   support

           0     1.0000    0.3077    0.4706        13
           1     0.9574    0.9045    0.9302       199
           2     0.9871    0.9889    0.9880       541
           3     0.7595    1.0000    0.8633        60

    accuracy                         0.9582       813
   macro avg     0.9260    0.8003    0.8130       813
weighted avg     0.9632    0.9582    0.9564       813

Confusion Matrix:
 [[  4   8   0   1]
 [  0 180   7  12]
 [  0   0 535   6]
 [  0   0   0  60]]

===== Model Metrics =====
Training Time: 27.47 seconds
Inference Time per Sample: 0.000597 seconds
Testing Time: 0.12 seconds
Model Size: 0.18 MB
Trainable Parameters: 41,408
RAM Usage During Inference: 2146.20 MB
